In [ ]:
# # Save a Notebook session:

# import dill
# dill.dump_session('notebook_env.db')

# # Restore a Notebook session:

# import dill
# dill.load_session('notebook_env.db')

In [ ]:
#%matplotlib notebook
import os
import re
import math
import sys
import igraph
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from tqdm.notebook import tqdm as tqdm
from scipy.cluster import hierarchy
from mpl_toolkits.mplot3d import Axes3D

### Code starts here

Looking for gsd files...

In [ ]:
gsd_file = "test_notebook/gel_l860_vfr0.3_vfp0_nG20_nR1170_nL390_k00_koff0.001_repuls500_bd1.0_Tc1.0_s1_dt0.002_gs0.001_N100000000.gsd"

In [ ]:
import glob
prefix = '_'.join(gsd_file.split('_')[:-1])
gsd_files = sorted( glob.glob(prefix + '_N*.gsd'),key=lambda x: x.split('_')[-1].replace('.gsd','').replace('N','') )
print(gsd_files)

In [ ]:
#can probably speed this up later:
def graph_distances(bonded_particles, particle_positions, box_size):
    distances = np.zeros(len(bonded_particles))
    for idx, pair in enumerate(bonded_particles):
        i,j = pair
        dr = particle_positions[j][1:4]-particle_positions[i][1:4]
        dr = dr-box_size*np.floor(dr/box_size+0.5)
        distances[idx] = np.sqrt((dr*dr).sum())
    return distances

#step time converts from MD steps to microseconds for this current setup
def get_trajectory_graph_info(gsd_files,analysis_stride=1, step_time=7.5e-2):
    from bond_analysis import bonds_analysis
    from gsd import hoomd as gsd
    trajectory = []
    for gsd_file in gsd_files:
        trajectory.extend(gsd.open(gsd_file,'rb')[1:]) # read gsd file
    box_length = None
    types = trajectory[0].particles.typeid
    N_A = int((types==0).sum())
    N_B = int((types==1).sum())
    n_binders = N_A+N_B
    assert N_A > N_B, "Warning, these may not be the right labels for A (n=%i) and B (n=%i)"%(N_A,N_B)
        
    time_list = []
    graph_list = []
    prev_positions = np.zeros((n_binders,3))
    deltaT_seconds = step_time/1e6
    for i in tqdm( range(0,len(trajectory),analysis_stride)):
        #for debugging:
        #if not i==100 and not i==101: continue
        step = trajectory[i].configuration.step
        box_size_i = trajectory[i].configuration.box[:3]
        if box_length is None:
            box_length = box_size_i[0]
        else:
            assert box_length == box_size_i[0], "error, box size is changing"

        bonded_particles, particle_positions, diffusivity = bonds_analysis(trajectory,frame_id=i,step_time=step_time/1e6)

        #skip frames before any bonding
        if(len(bonded_particles)==0):
            continue
        step_time_seconds = step*deltaT_seconds
        time_list.append(step_time_seconds)
        #particle positions is index, x,y,z, diffusivity
        g = igraph.Graph(n_binders, directed=False)
        edges = bonded_particles
        g.add_edges(edges)
        g.es['length'] = graph_distances(bonded_particles, particle_positions, box_size_i)

        g.vs['diffusivity'] = diffusivity
        g.vs['coordinate'] = particle_positions[:n_binders,1:4]
        graph_list.append(g)
    df = pd.DataFrame({
        'time': time_list,
        'graph': graph_list
    })
    df.L = box_length
    df.N_A = N_A
    df.N_B = N_B
    return df
 
   
df = get_trajectory_graph_info(gsd_files,analysis_stride=50)

Calculate the molecular concentration within random fixed volume changing with time ...

In [ ]:
L = df.L
N_A = df.N_A
N_B = df.N_B
r_box = L/4
r_center_list = np.array([[L/4,L/4,L/4],[-L/4,L/4,L/4],[L/4,-L/4,L/4],[-L/4,-L/4,L/4],[L/4,L/4,-L/4],[-L/4,L/4,-L/4],[L/4,-L/4,-L/4],[-L/4,-L/4,-L/4]]) #center of the investigate areas
N_center = np.zeros(shape=(len(r_center_list),len(df)))
concentrations = np.zeros(shape=(len(r_center_list),len(df)))
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    g = row['graph']
    Coord = g.vs['coordinate']
    for ridx, r_center in enumerate(r_center_list):
        dr = Coord - r_center
        n_in_box = ((np.abs(dr) < r_box).sum(axis=1)==3).sum()
        N_center[ridx,index] = n_in_box
        concentrations[ridx,index] = n_in_box*pow(10,7)/(pow(r_box*2,3)*6.02) #Calculate concentration within the cubic with uM unit

average_concentration = len(Coord)*pow(10,7)/(pow(L,3)*6.02)

        
fig, ax = plt.subplots(1,1, figsize=(6,4))
for i in range(len(r_center_list)):
    concentrations_i = concentrations[i,:]
    ax.plot(df.time,concentrations_i,label=f'center at {r_center_list[i]}')
ax.set_ylabel(f'Molecular concentration within cubic \n with length L/2={L/2} nm (µM)', fontsize=15)
ax.set_xlabel('Time (s)', fontsize=15)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.9))
ax.set_title(f'Dynamic of local molecular concentration')
ax.axhline(average_concentration,linestyle='--',color='black')
plt.show()


In [ ]:
df.graph

In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
from scipy.cluster import hierarchy
from tqdm.auto import tqdm

# Here the first part of nodes are dimers and remainings are hexamers
N_neighbor_time = []  # At each time points, number of neighbors of all nodes
giant_node = []  # At each time points, node index within the largest cluster
N_CONNECT_TOTAL = []  # Record number of nodes in the shortest path connecting two nodes in all time points
ORDER = []  # Record nodes order in the rearranged cluster graphs

plot_stride = 10
for gidx, g in enumerate(tqdm(df.graph)):
    N_neighbor = np.array([len(g.neighborhood(i)) - 1 for i in range(g.vcount())])
    if N_neighbor.sum() == 0:
        # skipping frames where there are no neighbors
        continue
    N_neighbor_time.append(N_neighbor)

    Direct_neighbor = np.zeros((g.vcount(), g.vcount()), dtype=int)
    for i in range(g.vcount()):
        Direct_neighbor[i, g.neighborhood(i)[1:]] = 1

    ccs = g.clusters()
    ccslistsize = np.array(ccs.sizes())
    giant_node.append(ccs[np.argmax(ccslistsize)])

    # Calculate shortest paths length between nodes
    spl_total = np.empty((g.vcount(), g.vcount()))
    N_connect_total = np.empty_like(spl_total)
    for v_source in range(g.vcount()):
        spl = np.array(g.shortest_paths_dijkstra(v_source, g.vs(), weights=g.es['length'])).squeeze()
        spl_total[v_source] = spl
        N_connect = np.array([len(i) for i in g.get_shortest_paths(v_source, g.vs())])
        N_connect_total[v_source] = N_connect

    # Replace infinities with a very large distance
    distances = spl_total.copy()
    distances[np.isinf(distances)] = distances[~np.isinf(distances)].max()

    # Clustering
    threshold = 1

    linkage = hierarchy.linkage(distance.squareform(distances), method="single")
    clusters = hierarchy.fcluster(linkage, threshold, criterion="distance")
    dend = hierarchy.dendrogram(linkage, color_threshold=threshold, no_plot=True)
    order = np.array(dend['leaves'])
    ORDER = np.append(ORDER, order, axis=0)

    distances = distances[order][:, order]
    N_connect_total = N_connect_total[order][:, order]
    Direct_neighbor = Direct_neighbor[order][:, order]
    if len(N_CONNECT_TOTAL) == 0:
        N_CONNECT_TOTAL = N_connect_total
    else:
        N_CONNECT_TOTAL = np.append(N_CONNECT_TOTAL, N_connect_total, axis=0)

    N_direct_neighbor = N_neighbor[order]
    N_direct_neighbor_matrix = np.tile(N_direct_neighbor[:, None], (1, g.vcount()))
    N_direct_neighbor_per = N_direct_neighbor / np.where(order < N_A, 2, 6)
    N_label = np.where(order < N_A, 1, 0)
    N_direct_neighbor_per_matrix = np.tile(N_direct_neighbor_per[:, None], (1, g.vcount()))
    N_label_matrix = np.tile(N_label[:, None], (1, g.vcount()))

#   Plot all the analysis results  
    #After clustering, convert the largest distances (& not indirectly connected nodes) to nan and plot as white in the colormap
    max_index = np.where(distances == np.amax(distances)) 
    distances[max_index] = np.nan
    no_connect_index = np.where(N_connect_total == 0)
    N_connect_total[no_connect_index] = np.nan
    
    if gidx%plot_stride == 0 or gidx==len(df.graph)-1:
        #skip plotting for now, uncomment next line
        #continue

        fig, axs = plt.subplots(1,5,figsize=(35,6))

        current_cmap = plt.cm.get_cmap()
        current_cmap.set_bad(color='white')

        [x,y] = np.where(Direct_neighbor == 1) #Extract x,y node position for those are direct neighbors

        sc0 = axs[0].imshow(distances)
    #     sc1 = axs[1].imshow(N_label_matrix)
        sc1 = axs[1].scatter(x, y, s=0.1) 
        sc2 = axs[2].imshow(N_direct_neighbor_per_matrix)
        sc3 = axs[3].imshow(N_connect_total)

        axs[0].invert_yaxis()
        axs[0].invert_xaxis()
        axs[1].invert_xaxis()
        axs[3].invert_yaxis()
        axs[3].invert_xaxis()


        cbar0 = fig.colorbar(sc0, ax=axs[0],extend='neither')
    #     sc0.set_clim(vmin=0,vmax=1.1)

        axs[1].set_xlim(0,Direct_neighbor.shape[0])
        axs[1].set_ylim(0,Direct_neighbor.shape[0])
        axs[1].set_aspect('equal')
    #     cbar1 = fig.colorbar(sc1, ax=axs[1],ticks=[0, 1])
    # #     cbar1.ax.set_yticklabels(['B', 'A'])
    #     cbar1.ax.set_yticklabels(['N','Y'])

        cbar2 = fig.colorbar(sc2, ax=axs[2],extend='neither')
    #     sc2.set_clim(vmin=0,vmax=1)

        cbar3 = fig.colorbar(sc3, ax=axs[3],extend='neither')
    #     sc3.set_clim(vmin=0,vmax=53)

        axs[2].xaxis.set_visible(False)
        cbar0.set_label(r'Topological shortest distance (nm)', fontsize=18)
    #     cbar1.set_label('Node type', fontsize=14)
        axs[1].yaxis.set_label_position('right')
        axs[1].set_ylabel('Direct Neighbors', fontsize=18)
        cbar2.set_label('precentage of direct neighbors', fontsize=18)
        cbar3.set_label('Number of nodes connecting path', fontsize=18)
        axs[0].set_title(f't = {df.time[gidx]:.3f} s', fontsize=18)    
        axs[3].set_title(f't = {df.time[gidx]:.3f} s', fontsize=18)    
        axs[4].set_xlabel("Node",fontsize=19)
    #     axs[4].set_ylabel("Dissimilarity")
    #     plt.subplots_adjust(wspace = 0.3)
    #    plt.show()

Calculate number of neighbors based on elements directly from graph or within largest cluster in each time frame

# Here the first part of nodes are dimers and remainings are hexamers
N_neighbor_time = [] #At each time points, number of neighbors of all nodes
giant_node = [] #At each time points, node index within the largest cluster
N_CONNECT_TOTAL = [] #Record number of nodes in the shortest path connecting two nodes in all time points
ORDER = [] #Record nodes order in the rearranged cluster graphs

#for index, row in tqdm(df.tail(n=5).iterrows(), total=5):
#for index, row in df.iterrows():#, total=df.shape[0]:
    #g = row['graph']
    #print(g)
plot_stride=10
for gidx, g in enumerate(tqdm(df.graph[::plot_stride])):
    N_neighbor = [len(g.neighborhood(i))-1 for i in range(0,g.vcount())] #The first value in the g.neighborhood is the node index itself
    if(np.sum(N_neighbor)==0): 
        #skipping frames where there are no neighbors
        continue
    N_neighbor_time.append(N_neighbor)
    
    Direct_neighbor = np.zeros((g.vcount(),g.vcount())) #Direct_neighbor[i,j] is 1 if (i,j) are direct neighbors
    for i in range(0,g.vcount()):
        Direct_neighbor[ i, g.neighborhood(i)[1:] ] = 1

    ccs = g.clusters()
    ccslistsize = list(ccs.sizes())
    giant_node.append(ccs[ccslistsize.index(max(ccslistsize))])
    
    # Calculate shortest paths length between nodes
    spl_total1 = []
    N_connect_total = []
    for v_source in range(0,g.vcount()):
        spl = g.shortest_paths_dijkstra(v_source, g.vs(), weights=g.es['length'])
        spl_total1.append(spl)
        N_connect = [float(len(i)) for i in g.get_shortest_paths(v_source, g.vs())] #Number of nodes connecting v_source and v_target (including v_source and v_target)
        N_connect_total.append(N_connect)
    distances = np.array(spl_total1).reshape((g.vcount(),g.vcount()))
    N_connect_total = np.array(N_connect_total).reshape((g.vcount(),g.vcount()))

    # Replace infinities with a very large distance
    distances[np.isinf(distances)] = distances[~np.isinf(distances)].max()
    
    # Clustering
    threshold = 1
    
    #skip plotting, comment next line
    fig, axs = plt.subplots(1,5,figsize=(35,6))
    
    sys.setrecursionlimit(10000) #Required for much larger system as trail20_(5-5) which has 1170 type A molecules
    linkage = hierarchy.linkage(distances, method="single")
    clusters = hierarchy.fcluster(linkage, threshold, criterion="distance")
    dend = hierarchy.dendrogram(linkage, color_threshold=threshold, no_plot=True)#ax=axs[4])
    order = dend['leaves']
    ORDER = np.append(ORDER,order,axis=0)
    distances = distances[order,:]
    distances = distances[:,order]
    N_connect_total = N_connect_total[order,:]
    N_connect_total = N_connect_total[:,order]
    Direct_neighbor = Direct_neighbor[order,:]
    Direct_neighbor = Direct_neighbor[:,order]
    if len(N_CONNECT_TOTAL)==0:
        N_CONNECT_TOTAL = N_connect_total
    else:
        N_CONNECT_TOTAL = np.append(N_CONNECT_TOTAL,N_connect_total,axis=0)

    
    #Calculate the direct neighbor numbers and the label for each nodes
    N_direct_neighbor = [N_neighbor[i] for i in order]
    N_direct_neighbor_matrix = np.tile(np.array([N_direct_neighbor]).transpose(),(1,np.int(g.vcount())))
    N_direct_neighbor_per = []
    N_label = []
    for i in range(0,len(order)):
        if order[i]<N_A:
            N_direct_neighbor_per.append(N_direct_neighbor[i]/2)
            N_label.append(1) #'A' is labled as 1
        else:
            N_direct_neighbor_per.append(N_direct_neighbor[i]/6)
            N_label.append(0) #'B' is labled as 0
    N_direct_neighbor_per_matrix = np.tile(np.array([N_direct_neighbor_per]).transpose(),(1,np.int(g.vcount())))
    N_label_matrix = np.tile(np.array([N_label]).transpose(),(1,np.int(g.vcount())))
#   Plot all the analysis results  
    #After clustering, convert the largest distances (& not indirectly connected nodes) to nan and plot as white in the colormap
    max_index = np.where(distances == np.amax(distances)) 
    distances[max_index] = np.nan
    no_connect_index = np.where(N_connect_total == 0)
    N_connect_total[no_connect_index] = np.nan
    
    #skip plotting for now, uncomment next line
    #continue

    
    current_cmap = plt.cm.get_cmap()
    current_cmap.set_bad(color='white')

    [x,y] = np.where(Direct_neighbor == 1) #Extract x,y node position for those are direct neighbors
    
    sc0 = axs[0].imshow(distances)
#     sc1 = axs[1].imshow(N_label_matrix)
    sc1 = axs[1].scatter(x, y, s=0.1) 
    sc2 = axs[2].imshow(N_direct_neighbor_per_matrix)
    sc3 = axs[3].imshow(N_connect_total)
    
    cbar0 = fig.colorbar(sc0, ax=axs[0],extend='neither')
#     sc0.set_clim(vmin=0,vmax=1.1)

    axs[1].set_xlim(0,Direct_neighbor.shape[0])
    axs[1].set_ylim(0,Direct_neighbor.shape[0])
    axs[1].invert_yaxis()
    axs[1].set_aspect('equal')
#     cbar1 = fig.colorbar(sc1, ax=axs[1],ticks=[0, 1])
# #     cbar1.ax.set_yticklabels(['B', 'A'])
#     cbar1.ax.set_yticklabels(['N','Y'])
    
    cbar2 = fig.colorbar(sc2, ax=axs[2],extend='neither')
#     sc2.set_clim(vmin=0,vmax=1)
    
    cbar3 = fig.colorbar(sc3, ax=axs[3],extend='neither')
#     sc3.set_clim(vmin=0,vmax=53)
    
    axs[2].xaxis.set_visible(False)
    cbar0.set_label(r'Topological shortest distance (nm)', fontsize=18)
#     cbar1.set_label('Node type', fontsize=14)
    axs[1].yaxis.set_label_position('right')
    axs[1].set_ylabel('Direct Neighbors', fontsize=18)
    cbar2.set_label('precentage of direct neighbors', fontsize=18)
    cbar3.set_label('Number of nodes connecting path', fontsize=18)
    axs[0].set_title(f't = {df.time[gidx*plot_stride]:.3f} s', fontsize=18)    
    axs[3].set_title(f't = {df.time[gidx*plot_stride]:.3f} s', fontsize=18)    
    axs[4].set_xlabel("Node",fontsize=19)
#     axs[4].set_ylabel("Dissimilarity")
#     plt.subplots_adjust(wspace = 0.3)
    plt.show()

Plot individual node clusters in 3d using coordinate values ...

R = [] #first cluster radius
MIU2_XY = [] #normalized first central moment on x,y direction, suggesting deviation from circular shape (0 suggests circular)
MIU2_XZ = [] #normalized first central moment on x,z direction
MIU2_YZ = [] #normalized first central moment on y,z direction
Cluster_size = [] #Number of nodes within the first tracked cluster at each time frames
#for index, row in tqdm(df.tail(n=5).iterrows(), total=5):
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    g = row['graph']
    Coord = g.vs['coordinate']
    order = ORDER[index*g.vcount():(index+1)*g.vcount()]
    n_connect_total = N_CONNECT_TOTAL[index*g.vcount():(index+1)*g.vcount()]
    ccs = g.clusters()
    N_cluster = len(ccs) #N_cluster is the number of clusters within each frame
    Ni_index = [] #Store the actual node index (not index directly from n_connect_total matrix) before clustering using order array
    check_temp_cluster = 0 #Checkpoint for the row index between different squares in graph
    if np.any(np.isnan(n_connect_total)): #whether N_CONNECT_TOTAL elements have been replace to NaN for plotting
        for j in range(0,N_cluster):
            temp_cluster = [i for i, e in enumerate(n_connect_total[check_temp_cluster]) if ~np.isnan(e)]
            Ni_index.append(order[temp_cluster])
            check_temp_cluster = check_temp_cluster+len(temp_cluster)
    else:
        for j in range(0,N_cluster):
            temp_cluster = [i for i, e in enumerate(n_connect_total[check_temp_cluster]) if e!=0]
            Ni_index.append(order[temp_cluster])
            check_temp_cluster = check_temp_cluster+len(temp_cluster)

    Len_Ni_index = [len(i) for i in Ni_index]
    
    #Find the cluster index that have the largest number of common node compared to the cluster in the previous frame, track first 3 clusters
    if index !=0:
        common_node_N1 = []
        common_node_N2 = []
        common_node_N3 = []
        for i_node in Ni_index:
            common_node_N1.append(len(set(N1_max_track_index_pre).intersection(set(i_node))))
            common_node_N2.append(len(set(N2_max_track_index_pre).intersection(set(i_node))))
            common_node_N3.append(len(set(N3_max_track_index_pre).intersection(set(i_node))))
        max_index_N1 = common_node_N1.index(max(common_node_N1)) #node cluster index that has largest number of common nodes with cluster in the previous timeframe
        max_index_N2 = common_node_N2.index(max(common_node_N2))
        max_index_N3 = common_node_N3.index(max(common_node_N3))

    #Find first five/six largest cluster within first 10 clusters at each time points
    Sort_Len = np.argsort(Len_Ni_index) #Returns the indices that would sort the array in ascending order
    N1_max_index = Ni_index[Sort_Len[len(Sort_Len)-1]]
    N2_max_index = Ni_index[Sort_Len[len(Sort_Len)-2]]
    N3_max_index = Ni_index[Sort_Len[len(Sort_Len)-3]]
    N4_max_index = Ni_index[Sort_Len[len(Sort_Len)-4]]
    N5_max_index = Ni_index[Sort_Len[len(Sort_Len)-5]]
#     N6_max_index = Ni_index[Sort_Len[len(Sort_Len)-6]]

    
    #Define the connected cluster at this current time frame and is used for comparison at next time frame, track first 3 clusters
    if index == 0:
        N1_max_track_index_pre = N1_max_index  #Choose the tracking cluster at first frame within range(0,N_cluster), here are the clusters among first five largest clusters in the first frame
        N2_max_track_index_pre = N2_max_index
        N3_max_track_index_pre = N3_max_index
    else: 
        N1_max_track_index_pre = Ni_index[max_index_N1]
        N2_max_track_index_pre = Ni_index[max_index_N2]
        N3_max_track_index_pre = Ni_index[max_index_N3]
    
    Cluster_size.append(len(N1_max_track_index_pre))
    
    #Plot scatter plot for the first five/six largest clusters at each time points
    fig = plt.figure(figsize=(14,5))
    ax1 = fig.add_subplot(1,2,1, projection='3d')
    N1_max_coord = [Coord[np.int(i)] for i in N1_max_index]
    N2_max_coord = [Coord[np.int(i)] for i in N2_max_index]
    N3_max_coord = [Coord[np.int(i)] for i in N3_max_index]
    N4_max_coord = [Coord[np.int(i)] for i in N4_max_index]
    N5_max_coord = [Coord[np.int(i)] for i in N5_max_index]
#     N6_max_coord = [Coord[np.int(i)] for i in N6_max_index]  
    
    #Caculate center of mass based on periodic boundary conditions through mapping x  (or y,z) dimension to a circle
    #(https://en.wikipedia.org/wiki/Center_of_mass#Systems_with_periodic_boundary_conditions)    
    theta = (np.array(N1_max_coord)+L/2)/L*2*math.pi  #Coordinates are from [0,L], thus plus L/2
    sin_theta = np.average(np.sin(theta),axis=0)
    cos_theta = np.average(np.cos(theta),axis=0)
    com_theta = np.array([math.atan2(-sin_theta[i],-cos_theta[i])+math.pi for i in range(3)])
    com = L*com_theta/2/math.pi-L/2 #Center of mass coordinates, substract L/2
#     print(np.average((N1_max_coord),axis=0))
#     print(com)
    
    r_vector = np.array(N1_max_coord)-com
    r_vector = r_vector - L*np.floor(r_vector/L+0.5)
    r = np.average([np.sqrt(np.power(i,2).sum()) for i in r_vector]) #or max(...) or np.median(...) or np.average(...)
    miu2_xy = sum([i[0]*i[1] for i in r_vector])/(len(r_vector)^2)
    miu2_xz = sum([i[0]*i[2] for i in r_vector])/(len(r_vector)^2)
    miu2_yz = sum([i[1]*i[2] for i in r_vector])/(len(r_vector)^2)
 
    R.append(r)
    MIU2_XY.append(miu2_xy)
    MIU2_XZ.append(miu2_xz)
    MIU2_YZ.append(miu2_yz)
    
    ax1.scatter([i[0] for i in N1_max_coord], [i[1] for i in N1_max_coord], [i[2] for i in N1_max_coord], c='r', marker='o', alpha=0.7, s=8, linewidths=0)
    ax1.scatter([i[0] for i in N2_max_coord], [i[1] for i in N2_max_coord], [i[2] for i in N2_max_coord], c='b', marker='o', alpha=0.7, s=8, linewidths=0)
    ax1.scatter([i[0] for i in N3_max_coord], [i[1] for i in N3_max_coord], [i[2] for i in N3_max_coord], c='g', marker='o', alpha=0.7, s=8, linewidths=0)    
    ax1.scatter([i[0] for i in N4_max_coord], [i[1] for i in N4_max_coord], [i[2] for i in N4_max_coord], c='c', marker='o', alpha=0.7, s=8, linewidths=0)    
    ax1.scatter([i[0] for i in N5_max_coord], [i[1] for i in N5_max_coord], [i[2] for i in N5_max_coord], c='m', marker='o', alpha=0.7, s=8, linewidths=0)    
#     ax1.scatter([i[0] for i in N6_max_coord], [i[1] for i in N6_max_coord], [i[2] for i in N6_max_coord], c='y', marker='o', alpha=0.7, s=8, linewidths=0)    
   
    ax1.set_title(f't = {df.time[index]:.3f} s')    
    ax1.set_zlim(-600,600)
    ticks = np.arange(-600, 600, 200)
    ax1.set_xticks(ticks)
    ax1.set_yticks(ticks)
    ax1.set_zticks(ticks)

    ax1.set_xlabel(r'Box X (nm)')
    ax1.set_ylabel(r'Box Y (nm)')
    ax1.set_zlabel(r'Box Z (nm)')
    
    #Plot scatter plot for the tracked clusters (the first three clusters in the 1st frame) over time 
    ax2 = fig.add_subplot(1,2,2, projection='3d')
    connected_coord_N1 = [Coord[np.int(i)] for i in N1_max_track_index_pre]
    connected_coord_N2 = [Coord[np.int(i)] for i in N2_max_track_index_pre]
    connected_coord_N3 = [Coord[np.int(i)] for i in N3_max_track_index_pre]
    ax2.scatter([i[0] for i in connected_coord_N1], [i[1] for i in connected_coord_N1], [i[2] for i in connected_coord_N1], c='r', marker='o', alpha=0.7, s=8, linewidths=0)
    ax2.scatter([i[0] for i in connected_coord_N2], [i[1] for i in connected_coord_N2], [i[2] for i in connected_coord_N2], c='b', marker='o', alpha=0.7, s=8, linewidths=0)
    ax2.scatter([i[0] for i in connected_coord_N3], [i[1] for i in connected_coord_N3], [i[2] for i in connected_coord_N3], c='g', marker='o', alpha=0.7, s=8, linewidths=0)
    ax2.set_title(f't = {df.time[index]:.3f} s')
    
    ax2.set_zlim(-600,600)
    ticks = np.arange(-600, 600, 200)
    ax2.set_xticks(ticks)
    ax2.set_yticks(ticks)
    ax2.set_zticks(ticks)

    ax2.set_xlabel(r'Box X (nm)')
    ax2.set_ylabel(r'Box Y (nm)')
    ax2.set_zlabel(r'Box Z (nm)')

    plt.show()

#Plot 0th and 1st moment with time for the largest cluster, indicating its actual radius and circularity in x/y/z plane
fig, axs = plt.subplots(1,2, figsize=(15,6))
axs[0].plot(df.time,R)
axs[0].set_ylabel(r'Radius of first cluster (nm)', fontsize=18)
axs[0].set_xlabel('Time (s)', fontsize=18)
# axs[0].set_ylim(0,0.18)

axs[1].plot(df.time,MIU2_XY,color='blue', label='X,Y direction')
axs[1].plot(df.time,MIU2_XZ,color='red', label='X,Z direction')
axs[1].plot(df.time,MIU2_YZ,color='green', label='Y,Z direction')
axs[1].set_ylabel('Normalized first central moment', fontsize=18)
axs[1].set_xlabel('Time (s)', fontsize=18)
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.9))
# axs[1].set_ylim(-0.012,0.012)

#Plot the number of nodes within the first tracked cluster over timed
dt = np.diff(df.time)
dCs = np.diff(Cluster_size)
fig,axs = plt.subplots(1,2, figsize=(15,6))
axs[0].plot(df.time,Cluster_size)
axs[0].set_ylabel(r'Number of nodes within first tracked cluster', fontsize=18)
axs[0].set_xlabel('Time (s)', fontsize=18)
# ax[0].set_ylim(0,280)

axs[1].plot(df.time[1:],[x/y for x, y in zip(dCs, dt)])
axs[1].set_ylabel(r'First derivation of node number', fontsize=18)
axs[1].set_xlabel('Time (s)', fontsize=18)
plt.show()

### Added Cell 1: Plot individual node clusters in 3d using coordinate values by plotting 5 or 6 largest clusters at each time points or tracked clusters from first 3 largest clusters in the last frame..

In [ ]:
R = [] #first cluster radius
MIU2_XY = [] #normalized first central moment on x,y direction, suggesting deviation from circular shape (0 suggests circular)
MIU2_XZ = [] #normalized first central moment on x,z direction
MIU2_YZ = [] #normalized first central moment on y,z direction
Cluster_size = [] #Number of nodes within the first tracked cluster at each time frames

df_copy = df.copy()  #Copy dataframe 'df' to a new dataframe 'df_copy' to avoid changing original dataframe
reversed_df = df_copy.loc[::-1] #Reverse copied dataframe in terms of index and saved to 'reversed_df' dataframe
# for index, row in tqdm(reversed_df.head(n=5).iterrows(), total=5):
for index, row in tqdm(reversed_df.iterrows(), total=reversed_df.shape[0]):
    if not (index%plot_stride == 0): continue
    if index==len(reversed_df)-1: continue
    
    g = row['graph']
    Coord = g.vs['coordinate']
    order = ORDER[index*g.vcount():(index+1)*g.vcount()]
    n_connect_total = N_CONNECT_TOTAL[index*g.vcount():(index+1)*g.vcount()]
    
    ccs = g.clusters()
    N_cluster = len(ccs) #N_cluster is the number of clusters within each frame
    Ni_index = [] #Store the actual node index (not index directly from n_connect_total matrix) before clustering using order array
    check_temp_cluster = 0 #Checkpoint for the row index between different squares in graph
    if np.any(np.isnan(n_connect_total)): #whether N_CONNECT_TOTAL elements have been replace to NaN for plotting
        for j in range(0,N_cluster):
            temp_cluster = [i for i, e in enumerate(n_connect_total[check_temp_cluster]) if ~np.isnan(e)]
            Ni_index.append(order[temp_cluster])
            check_temp_cluster = check_temp_cluster+len(temp_cluster)
    else:
        for j in range(0,N_cluster):
            temp_cluster = [i for i, e in enumerate(n_connect_total[check_temp_cluster]) if e!=0]
            Ni_index.append(order[temp_cluster])
            check_temp_cluster = check_temp_cluster+len(temp_cluster)

    Len_Ni_index = [len(i) for i in Ni_index]
    
    #Find the cluster index that have the largest number of common node compared to the cluster in the previous frame, track first 3 clusters
    if index != reversed_df.shape[0]-1:
        common_node_N1 = []
        common_node_N2 = []
        common_node_N3 = []
        for i_node in Ni_index:
            common_node_N1.append(len(set(N1_max_track_index_pre).intersection(set(i_node))))
            common_node_N2.append(len(set(N2_max_track_index_pre).intersection(set(i_node))))
            common_node_N3.append(len(set(N3_max_track_index_pre).intersection(set(i_node))))
        max_index_N1 = common_node_N1.index(max(common_node_N1)) #node cluster index that has largest number of common nodes with cluster in the previous timeframe
        max_index_N2 = common_node_N2.index(max(common_node_N2))
        max_index_N3 = common_node_N3.index(max(common_node_N3))

    #Find first five/six largest cluster at each time points
    Sort_Len = np.argsort(Len_Ni_index) #Returns the indices that would sort the array in ascending order
    N1_max_index = Ni_index[Sort_Len[len(Sort_Len)-1]]
    N2_max_index = Ni_index[Sort_Len[len(Sort_Len)-2]]
    N3_max_index = Ni_index[Sort_Len[len(Sort_Len)-3]]
    N4_max_index = Ni_index[Sort_Len[len(Sort_Len)-4]]
    N5_max_index = Ni_index[Sort_Len[len(Sort_Len)-5]]
#     N6_max_index = Ni_index[Sort_Len[len(Sort_Len)-6]]

    
    #Define the connected cluster at this current time frame and is used for comparison at next time frame, track first 3 clusters
    if index == reversed_df.shape[0]-1:
        N1_max_track_index_pre = N1_max_index  #Choose the tracking cluster at first frame within range(0,N_cluster), here are the clusters among first five largest clusters in the first frame
        N2_max_track_index_pre = N2_max_index
        N3_max_track_index_pre = N3_max_index
    else: 
        N1_max_track_index_pre = Ni_index[max_index_N1]
        N2_max_track_index_pre = Ni_index[max_index_N2]
        N3_max_track_index_pre = Ni_index[max_index_N3]
    
    Cluster_size.append(len(N1_max_track_index_pre))
    
    #Plot scatter plot for the first five/six largest clusters at each time points
    fig = plt.figure(figsize=(14,5))
    ax1 = fig.add_subplot(1,2,1, projection='3d')
    N1_max_coord = [Coord[np.int(i)] for i in N1_max_index]
    N2_max_coord = [Coord[np.int(i)] for i in N2_max_index]
    N3_max_coord = [Coord[np.int(i)] for i in N3_max_index]
    N4_max_coord = [Coord[np.int(i)] for i in N4_max_index]
    N5_max_coord = [Coord[np.int(i)] for i in N5_max_index]
#     N6_max_coord = [Coord[np.int(i)] for i in N6_max_index]  
    
    #Caculate center of mass based on periodic boundary conditions through mapping x  (or y,z) dimension to a circle
    #(https://en.wikipedia.org/wiki/Center_of_mass#Systems_with_periodic_boundary_conditions)    
    theta = (np.array(N1_max_coord)+L/2)/L*2*math.pi  #Coordinates are from [0,L], thus plus L/2
    sin_theta = np.average(np.sin(theta),axis=0)
    cos_theta = np.average(np.cos(theta),axis=0)
    com_theta = np.array([math.atan2(-sin_theta[i],-cos_theta[i])+math.pi for i in range(3)])
    com = L*com_theta/2/math.pi-L/2 #Center of mass coordinates, substract L/2
#     print(np.average((N1_max_coord),axis=0))
#     print(com)
    
    r_vector = np.array(N1_max_coord)-com
    r_vector = r_vector - L*np.floor(r_vector/L+0.5)
    r = np.average([np.sqrt(np.power(i,2).sum()) for i in r_vector]) #or max(...) or np.median(...) or np.average(...)
    miu2_xy = sum([i[0]*i[1] for i in r_vector])/(len(r_vector)^2)
    miu2_xz = sum([i[0]*i[2] for i in r_vector])/(len(r_vector)^2)
    miu2_yz = sum([i[1]*i[2] for i in r_vector])/(len(r_vector)^2)
 
    R.append(r)
    MIU2_XY.append(miu2_xy)
    MIU2_XZ.append(miu2_xz)
    MIU2_YZ.append(miu2_yz)
    
    ax1.scatter([i[0] for i in N1_max_coord], [i[1] for i in N1_max_coord], [i[2] for i in N1_max_coord], c='r', marker='o', alpha=0.7, s=8, linewidths=0)
    ax1.scatter([i[0] for i in N2_max_coord], [i[1] for i in N2_max_coord], [i[2] for i in N2_max_coord], c='b', marker='o', alpha=0.7, s=8, linewidths=0)
    ax1.scatter([i[0] for i in N3_max_coord], [i[1] for i in N3_max_coord], [i[2] for i in N3_max_coord], c='g', marker='o', alpha=0.7, s=8, linewidths=0)    
    ax1.scatter([i[0] for i in N4_max_coord], [i[1] for i in N4_max_coord], [i[2] for i in N4_max_coord], c='c', marker='o', alpha=0.7, s=8, linewidths=0)    
    ax1.scatter([i[0] for i in N5_max_coord], [i[1] for i in N5_max_coord], [i[2] for i in N5_max_coord], c='m', marker='o', alpha=0.7, s=8, linewidths=0)    
#     ax1.scatter([i[0] for i in N6_max_coord], [i[1] for i in N6_max_coord], [i[2] for i in N6_max_coord], c='y', marker='o', alpha=0.7, s=8, linewidths=0)    
   
    ax1.set_title(f't = {reversed_df.time[index]:.3f} s')    
    ax1.set_zlim(-600,600)
    ticks = np.arange(-600, 600, 200)
    ax1.set_xticks(ticks)
    ax1.set_yticks(ticks)
    ax1.set_zticks(ticks)

    ax1.set_xlabel(r'Box X (nm)')
    ax1.set_ylabel(r'Box Y (nm)')
    ax1.set_zlabel(r'Box Z (nm)')
    
    #Plot scatter plot for the tracked clusters (the first three clusters in the last frame) over time 
    ax2 = fig.add_subplot(1,2,2, projection='3d')
    connected_coord_N1 = [Coord[np.int(i)] for i in N1_max_track_index_pre]
    connected_coord_N2 = [Coord[np.int(i)] for i in N2_max_track_index_pre]
    connected_coord_N3 = [Coord[np.int(i)] for i in N3_max_track_index_pre]
    ax2.scatter([i[0] for i in connected_coord_N1], [i[1] for i in connected_coord_N1], [i[2] for i in connected_coord_N1], c='r', marker='o', alpha=0.7, s=8, linewidths=0)
    ax2.scatter([i[0] for i in connected_coord_N2], [i[1] for i in connected_coord_N2], [i[2] for i in connected_coord_N2], c='b', marker='o', alpha=0.7, s=8, linewidths=0)
    ax2.scatter([i[0] for i in connected_coord_N3], [i[1] for i in connected_coord_N3], [i[2] for i in connected_coord_N3], c='g', marker='o', alpha=0.7, s=8, linewidths=0)
    ax2.set_title(f't = {reversed_df.time[index]:.3f} s')
    
    ax2.set_zlim(-600,600)
    ticks = np.arange(-600, 600, 200)
    ax2.set_xticks(ticks)
    ax2.set_yticks(ticks)
    ax2.set_zticks(ticks)

    ax2.set_xlabel(r'Box X (nm)')
    ax2.set_ylabel(r'Box Y (nm)')
    ax2.set_zlabel(r'Box Z (nm)')

    plt.show()

#Plot 0th and 1st moment with time for the largest cluster, indicating its actual radius and circularity in x/y/z plane
fig, axs = plt.subplots(1,2, figsize=(15,6))
axs[0].plot(reversed_df.time,R)
axs[0].set_ylabel(r'Radius of first cluster (nm)', fontsize=18)
axs[0].set_xlabel('Time (s)', fontsize=18)
# axs[0].set_ylim(0,0.18)

axs[1].plot(reversed_df.time,MIU2_XY,color='blue', label='X,Y direction')
axs[1].plot(reversed_df.time,MIU2_XZ,color='red', label='X,Z direction')
axs[1].plot(reversed_df.time,MIU2_YZ,color='green', label='Y,Z direction')
axs[1].set_ylabel('Normalized first central moment', fontsize=18)
axs[1].set_xlabel('Time (s)', fontsize=18)
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.9))
# axs[1].set_ylim(-0.012,0.012)

##Plot the number of nodes within the first tracked cluster over timed
#dt = np.diff(reversed_df.time)
#dCs = np.diff(Cluster_size)
#fig,axs = plt.subplots(1,2, figsize=(15,6))
#axs[0].plot(reversed_df.time,Cluster_size)
#axs[0].set_ylabel(r'Number of nodes within first tracked cluster', fontsize=18)
#axs[0].set_xlabel('Time (s)', fontsize=18)
## ax[0].set_ylim(0,280)

#axs[1].plot(df.time[1:],[x/y for x, y in zip(dCs, dt)])
#axs[1].set_ylabel(r'First derivation of node number', fontsize=18)
#axs[1].set_xlabel('Time (s)', fontsize=18)
#plt.show()

Calculate the neighbor properties of all nodes as well as the nodes within the largest cluster...

In [ ]:
indexA = np.array(range(N_A)) #Node index number for typeA molecules
indexB = np.array(range(N_A,N_A+N_B)) #Node index number for typeB molecules

N_neighbor_node = [] #For each node, number of neighbors at all time points
N_neighbor_node_percen = [] #For each node, percentage of neighbors number relative to all available binding sites
# node_label = [None]*len(N_neighbor_time[0]) #initialize the node_label with total number of A/B nodes
for node_index in range(0,len(N_neighbor_time[0])):
    N_neighbor_indi_node = [N_neighbor_time[time_index][node_index] for time_index in range(0,len(N_neighbor_time))]
    #number of neighbors for each individual nodes at all time points
    N_neighbor_node.append(N_neighbor_indi_node)
    if node_index in indexA:
#         node_label[node_index] = 'A'
        N_neighbor_node_percen.append([x/2 for x in N_neighbor_indi_node])
    else:
#         node_label[node_index] = 'B'
        N_neighbor_node_percen.append([x/6 for x in N_neighbor_indi_node])

    
N_neighbor_node_giant = [] #Initialize average neighbor numbers for nodes within largest cluster
N_neighbor_node_giant_std = [] #std for neighbor numbers of nodes within largest cluster
for time_index in range(0,len(giant_node)):
    N_giant_node = giant_node[time_index]
    N_neighbor_tempA = []
    N_neighbor_tempB = []
    for i in N_giant_node: 
        if i in indexA:
            N_neighbor_tempA.append(N_neighbor_node[i][time_index])
        else:
            N_neighbor_tempB.append(N_neighbor_node[i][time_index])
    N_neighbor_node_giant.append([np.average(N_neighbor_tempA),np.average(N_neighbor_tempB)])
    N_neighbor_node_giant_std.append([np.std(N_neighbor_tempA),np.std(N_neighbor_tempB)])

Plot number of neighbors for all nodes or nodes within largest cluster over time ...

In [ ]:
# Plot number of neighbors for all nodes over time
num_frames = len(df)
maxlags = min(num_frames-1,100)

fig, ax = plt.subplots(1,1, figsize=(6,4))
# ax.set_ylim(0,6.5)
ax.plot(df.time,np.average([N_neighbor_node[i] for i in indexA],axis=0), color='blue',label='dimer neighbors')
ax.plot(df.time,np.average([N_neighbor_node[i] for i in indexB],axis=0), color='red',label='hexamer neighbors')
ax.fill_between(
    df.time,
    y1 = np.average([N_neighbor_node[i] for i in indexA],axis=0)-np.std([N_neighbor_node[i] for i in indexA],axis=0),
    y2 = np.average([N_neighbor_node[i] for i in indexA],axis=0)+np.std([N_neighbor_node[i] for i in indexA],axis=0), alpha=0.3, facecolor='gray')

ax.fill_between(
    df.time,
    y1 = np.average([N_neighbor_node[i] for i in indexB],axis=0)-np.std([N_neighbor_node[i] for i in indexB],axis=0),
    y2 = np.average([N_neighbor_node[i] for i in indexB],axis=0)+np.std([N_neighbor_node[i] for i in indexB],axis=0), alpha=0.3, color='gray')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.9))
ax.set_ylabel('Number of neighbors', fontsize=18)
ax.set_xlabel('Time (s)', fontsize=18)
plt.title('Average of all nodes')
plt.show()

# Plot number of neighbors for nodes within largest cluster over time
fig, ax = plt.subplots(1,1, figsize=(6,4))
# ax.set_ylim(0,6.5)
ax.plot(df.time,[N_neighbor_node_giant[i][0] for i in range(0,len(N_neighbor_node_giant))], color='blue',label='dimer neighbors')
ax.plot(df.time,[N_neighbor_node_giant[i][1] for i in range(0,len(N_neighbor_node_giant))], color='red',label='hexamer neighbors')
ax.fill_between(
    df.time,
    y1 = [N_neighbor_node_giant[i][0]-N_neighbor_node_giant_std[i][0] for i in range(0,len(N_neighbor_node_giant))],
    y2 = [N_neighbor_node_giant[i][0]+N_neighbor_node_giant_std[i][0] for i in range(0,len(N_neighbor_node_giant))], alpha=0.3, facecolor='gray')

ax.fill_between(
    df.time,
    y1 = [N_neighbor_node_giant[i][1]-N_neighbor_node_giant_std[i][1] for i in range(0,len(N_neighbor_node_giant))],
    y2 = [N_neighbor_node_giant[i][1]+N_neighbor_node_giant_std[i][1] for i in range(0,len(N_neighbor_node_giant))], alpha=0.3, facecolor='gray')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.9))
ax.set_ylabel('Number of neighbors', fontsize=18)
ax.set_xlabel('Time (s)', fontsize=18)
plt.title('Average of the nodes within largest cluster')
plt.show()

# Plot autocorrelation of number of neighbors for all nodes over time
fig, axes = plt.subplots(1,2, figsize=(12,4), sharey=True)
# ax.set_ylim(0,6.5)
axes[0].acorr(np.average([N_neighbor_node[i] for i in indexA],axis=0), maxlags = maxlags, color='blue',label='dimer neighbors')
axes[1].acorr(np.average([N_neighbor_node[i] for i in indexB],axis=0), maxlags = maxlags, color='red',label='hexamer neighbors') 
# axes.legend(loc='center left', bbox_to_anchor=(1, 0.9))
axes[0].set_ylabel(f'Autocorrelation of \n node neighbor numbers', fontsize=18)
axes[0].set_xlabel('Time lag', fontsize=18)
axes[1].set_xlabel('Time lag', fontsize=18)
axes[0].set_title('Average of all dimer neighbors')
axes[1].set_title('Average of all hexamer neighbors')
plt.show()

# Plot autocorrelation of number of neighbors of nodes within largest cluster in the last frame over time
fig, axes = plt.subplots(1,2, figsize=(12,4), sharey=True)
Final_giant_node = giant_node[-1]
Final_giant_neighbor_A = [] #For each typeA node (dimer) within last giant cluster, number of neighbors at all time points
Final_giant_neighbor_B = [] #For each typeB node (hexamer) within last giant cluster, number of neighbors at all time points
for node_index in Final_giant_node:
    if node_index in indexA:
        Final_giant_neighbor_A.append(N_neighbor_node[node_index])
    else:
        Final_giant_neighbor_B.append(N_neighbor_node[node_index])
axes[0].acorr(np.average(Final_giant_neighbor_A,axis=0), maxlags = maxlags, color='blue',label='dimer neighbors')
axes[1].acorr(np.average(Final_giant_neighbor_B,axis=0), maxlags = maxlags, color='red',label='hexamer neighbors') 
# axes.legend(loc='center left', bbox_to_anchor=(1, 0.9))
axes[0].set_ylabel(f'Autocorrelation of \n node neighbor numbers \n within lart largest giant cluster', fontsize=18)
axes[0].set_xlabel('Time lag', fontsize=18)
axes[1].set_xlabel('Time lag', fontsize=18)
axes[0].set_title(f'Average of dimer neighbors \n within last largest giant cluster')
axes[1].set_title(f'Average of hexamer neighbors \n within last largest giant cluster')
plt.show()

In [ ]:


# Plot colormap distribution of percentage neighbors and diffusivity for nodes within largest cluster
cmap = plt.cm.get_cmap('jet')
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
# for index, row in tqdm(df.head(n=5).iterrows(), total=5):
    if not (index%plot_stride == 0 or index==len(df.graph)-1): continue

    g = row['graph']
    Coord = g.vs['coordinate']
    N_giant_node = giant_node[index]
    N_giant_coord = [Coord[i] for i in N_giant_node]
    N_giant_node_percen = [N_neighbor_node_percen[i][index] for i in N_giant_node]
    N_giant_diffusion = [g.vs['diffusivity'][i] for i in N_giant_node]

    N_giant_node_A = []
    N_giant_node_B = []
    for i in range(0, len(N_giant_node)):
        if N_giant_node[i] in indexA:
            N_giant_node_A.append(N_giant_node[i])
        else:
            N_giant_node_B.append(N_giant_node[i])
    N_giant_coord_A = [Coord[i] for i in N_giant_node_A]
    N_giant_coord_B = [Coord[i] for i in N_giant_node_B]
    N_giant_node_percen_A = [N_neighbor_node_percen[i][index] for i in N_giant_node_A]
    N_giant_node_percen_B = [N_neighbor_node_percen[i][index] for i in N_giant_node_B]
    N_giant_diffusion_A = [g.vs['diffusivity'][i] for i in N_giant_node_A]
    N_giant_diffusion_B = [g.vs['diffusivity'][i] for i in N_giant_node_B]
    
    #First subplot - 3d node images
    fig = plt.figure(figsize=(30,4))
    gs = gridspec.GridSpec(1, 5, width_ratios=[1.5, 1.2, 1.2, 0.8, 0.8]) 
    ax3D = fig.add_subplot(gs[0], projection='3d')
    sc3D = ax3D.scatter([i[0] for i in N_giant_coord], [i[1] for i in N_giant_coord], [i[2] for i in N_giant_coord], c=[i for i in N_giant_node_percen], cmap=cmap, marker='o', alpha=0.7, s=8, linewidths=0, depthshade=0)    
    ax3D.set_title(f't = {df.time[index]:.3f} s')    
    cbar = plt.colorbar(sc3D,extend='neither')
    sc3D.set_clim(vmin=0,vmax=1)
    cbar.set_label('percentage of direct neighbors', fontsize=14)
    
    def forceUpdate(event): #Solve the problem of point color changing in 3d scatter plot compared to 2d scatter plot
        global sc3D
        sc3D.changed()
    fig.canvas.mpl_connect('draw_event', forceUpdate)

    ax3D.set_xlim(-500,500)
    ax3D.set_ylim(-500,500)
    ax3D.set_zlim(-500,500)
    ticks = np.arange(-500, 500, 200)
    ax3D.set_xticks(ticks)
    ax3D.set_yticks(ticks)
    ax3D.set_zticks(ticks)

    ax3D.set_xlabel(r'Box X (nm)')
    ax3D.set_ylabel(r'Box Y (nm)')
    ax3D.set_zlabel(r'Box Z (nm)')
    
    #Second subplot - projection on x/y aixs node images with noder neighbor percentage as colorcode
    ax = fig.add_subplot(gs[1])
    sc = ax.scatter([i[0] for i in N_giant_coord], [i[1] for i in N_giant_coord], c=[i for i in N_giant_node_percen], cmap=cmap, marker='o', s=5, linewidths=0)    
    ax.set_title(f't = {df.time[index]:.3f} s')    
    cbar = plt.colorbar(sc,extend='neither')
    sc.set_clim(vmin=0,vmax=1)
    cbar.set_label('percentage of direct neighbors', fontsize=14)

    ax.set_xlim(-500,500)
    ax.set_ylim(-500,500)
    ticks = np.arange(-500, 500, 200)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    ax.set_xlabel(r'Box X (nm)')
    ax.set_ylabel(r'Box Y (nm)')
    
    #Third subplot - projection on x/y aixs node images with diffusivity as colorcode
    ax = fig.add_subplot(gs[2])
    sc = ax.scatter([i[0] for i in N_giant_coord], [i[1] for i in N_giant_coord], c=[g.vs['diffusivity'][i] for i in N_giant_node], cmap=cmap, marker='o', s=5, linewidths=0)    
    ax.set_title(f't = {df.time[index]:.3f} s')    
    cbar = plt.colorbar(sc,extend='neither')
#     sc.set_clim(vmin=0,vmax=0.25)
    cbar.set_label(r'Avg. node diffusivity ($\mu m^2/s$)', fontsize=14)

    ax.set_xlim(-500,500)
    ax.set_ylim(-500,500)
    ticks = np.arange(-500, 500, 200)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    ax.set_xlabel(r'Box X (nm)')
    ax.set_ylabel(r'Box Y (nm)')
    
    #Forth subplot - A/B node separation projection on x/y aixs
    ax = fig.add_subplot(gs[3])
    sc = ax.scatter([i[0] for i in N_giant_coord_A], [i[1] for i in N_giant_coord_A], c='m', cmap=cmap, marker='o', label='A: Dimer', s=5,alpha=0.5)    
    sc = ax.scatter([i[0] for i in N_giant_coord_B], [i[1] for i in N_giant_coord_B], c='y', cmap=cmap, marker='o', label='B: Hexamer', s=5, alpha=0.5)    
    ax.set_title(f't = {df.time[index]:.3f} s')    

#     ax.legend(loc='center left', bbox_to_anchor=(1, 0.9))

    ax.set_xlim(-500,500)
    ax.set_ylim(-500,500)
    ticks = np.arange(-500, 500, 200)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    ax.set_xlabel(r'Box X (nm)')
    ax.set_ylabel(r'Box Y (nm)')

    #Fifth subplot - Plots of number of neighbors for each node vs. node diffusivity
    ax = fig.add_subplot(gs[4])
    sc = ax.plot(N_giant_node_percen_A, N_giant_diffusion_A, 'mo', label='A: Dimer',alpha=0.5)    
    sc = ax.plot(N_giant_node_percen_B, N_giant_diffusion_B, 'yo', label='B: Hexamer', alpha=0.5)    
    ax.set_title(f't = {df.time[index]:.3f} s')    

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.9))
    ax.set_ylim(-0.005,0.25)

    ax.set_xlabel(r'Percentage of node neighbors')
    ax.set_ylabel(r'Avg. node diffusivity ($\mu m^2/s$)')
    
    plt.show()
    

Calculating topological properties of largest cluster...

In [ ]:
df['others_size'] = None

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        
    g = row['graph']
    
    # Calculate connected compoents (CCs)
    ccs = g.clusters()
    
    # Get giant component
    giant = ccs.giant()

#     # Size of giant component
    df.loc[index,'giant_size'] = giant.vcount()

    # Size of giant component in terms of #connections
#     df.loc[index,'giant_size'] = giant.ecount()
    
    # Diameter of giant connected component
    # Diameters is defined as the longest shortest path
    # between two pairs of nodes
#     df.loc[index,'giant_diameter'] = giant.diameter()
    df.loc[index,'giant_diameter'] = giant.diameter(weights=giant.es['length'])
    
    # Std and mean of sizes of remaining components
    sizes = ccs.sizes()
#     sizes = [g.ecount() for g in ccs.subgraphs()]

    # Exclude giant cc
    sizes.pop(sizes.index(giant.vcount()))
#     sizes.pop(sizes.index(giant.ecount()))
    df.loc[index,'mean_others_size'] = np.mean(sizes) if sizes else None
    df.loc[index,'std_others_size'] = np.std(sizes) if sizes else None
    
    # Frequency of clusters with size 0, 1, 2..
    hist = np.bincount(sizes)
    df.at[index,'others_size'] = hist

Show results...

In [ ]:
# ymax = 1.1*df.giant_size.max()
fig, ax1 = plt.subplots(1,1, figsize=(6,4))
ax1.plot(df.time,df.giant_size, color='blue', label='Number of nodes within largest cluster')
ax1.set_ylabel("Number of nodes \n within largest cluster", fontsize=18, color='blue')
ax1.set_xlabel('Time (s)', fontsize=18)
# ax1.set_ylim(0,480)
plt.show()

fig, ax2 = plt.subplots(1,1, figsize=(6,4))
ax2.set_ylabel('Topological diameter \n of largest cluster (nm)', fontsize=18, color='red')
ax2.plot(df.time,df.giant_diameter, color='red', label='Topological diameter of largest cluster')
# ax2.set_ylim(0,1.2)
# ax1.legend(loc='center left', bbox_to_anchor=(1.15, 0.9))
# ax2.legend(loc='center left', bbox_to_anchor=(1.15, 0.8))
plt.show
# fig.savefig('./Ensemble-node-links/box_Viscosity_trail19_(1)_nodelinks/giant cluster trail19 (1).pdf')

#Plot first derivative of nodes number within largest cluster vs. time
dt = np.diff(df.time)
dGDs = np.diff(df.giant_size)
fig, ax = plt.subplots(1,1, figsize=(6,4))
ax.plot(df.time[1:],[x/y for x, y in zip(dGDs, dt)])
ax.set_ylabel("First derivative of node number \n within largest cluster", fontsize=18)
ax.set_xlabel('Time (s)', fontsize=18)
plt.show()

Creating a dataframe with time and file paths for links and coordinates...

In [ ]:
xmax = df.time.max()
ymax = 1.1*df.giant_size.max()
fig, ax = plt.subplots(1,1, figsize=(6,4))
ax.set_ylabel('Size of remaining CCs', fontsize=18)
ax.set_xlabel('Time (s)', fontsize=18)
ax.set_xlim(0,xmax)
# ax.set_ylim(0,30)
df_sub = df.dropna()
ax.fill_between(
    df_sub.time,
    y1 = df_sub.mean_others_size-df_sub.std_others_size,
    y2 = df_sub.mean_others_size+df_sub.std_others_size, alpha=0.3, color='gray')
ax.plot(df_sub.time,df_sub.mean_others_size, '-o', color='black')
plt.show()

In [ ]:
# Evolution of clusters sizes as a heatmap
# This does not include the giant cluster

# Calculate the maximum number of co-occurent clusters
nmax = []
for index in df.index:
    hist = df.others_size[index]
    if hist.size:
        nmax.append(len(hist))
# Empty heatmap
data = np.zeros((np.max(nmax),df.shape[0]), dtype=np.uint64)
# Fill heatmap in
for i, index in enumerate(df.index):
    hist = df.others_size[index]
    data[:len(hist),i] = hist
fig, ax = plt.subplots(1,1, figsize=(12,4))
sc0 = ax.imshow(np.log(1+data[:40,:]),cmap='jet')
# sc0 = ax.imshow(data,cmap='jet')
# ax.set_xlabel(f'Time (s x {df.shape[0]})', fontsize=18)
# ax.set_xlabel('Time (s x %1.2f)' %np.diff(df_sub.time)[0], fontsize=18)
ax.set_xlabel(f'Time (x {np.diff(df_sub.time)[0]:.3f} s)', fontsize=18)
ax.set_ylabel('Cluster size', fontsize=18)
clbar = fig.colorbar(sc0,extend='neither')
# sc0.set_clim(vmin=0,vmax=6.8)
clbar.set_label('ln(1 + #Clusters)',fontsize=14)
plt.show()

Fraction of neighboors that remains the same

In [ ]:
for idxi, idxj in tqdm(zip(df.index[:-1],df.index[1:]), total=df.shape[0]-1):
    
    # Load graph at time   t: gi
    # Load graph at time t+1: gj
    gi = df.graph[idxi]
    gj = df.graph[idxj]
    
    # Get list of neighbors
    neighi = gi.neighborhood()
    neighj = gj.neighborhood()
        
    fraction = []
    
    # Check the fraction of nodes with unchanged neighborhood
    for nik, njk in zip(neighi,neighj):
        
        # Intersection between the two sets of neighs
        common = set(njk).intersection(set(nik))
        
        fraction.append(len(common) / np.max([len(nik),len(njk)]))
        
    df.loc[idxj,'frac_sim_neighs_avg'] = np.mean(fraction)
    df.loc[idxj,'frac_sim_neighs_std'] =  np.std(fraction)

# Plot results
xmax = df.time.max()
fig, ax = plt.subplots(1,1, figsize=(6,4))
ax.set_ylabel('Fraction of unchanged neighbors', fontsize=18)
ax.set_xlabel('Time (s)', fontsize=18)
ax.set_xlim(0,xmax)
# ax.set_ylim(0.6,1.1)
ax.fill_between(
    df.time,
    y1 = df.frac_sim_neighs_avg-df.frac_sim_neighs_std,
    y2 = df.frac_sim_neighs_avg+df.frac_sim_neighs_std, alpha=0.3, color='gray')
ax.plot(df.time,df.frac_sim_neighs_avg, '-o', color='black')

# Plot zoom-in results from 0-0.25s
fig, ax_zoom = plt.subplots(1,1, figsize=(1,4))
# ax_zoom.set_ylabel('Fraction of unchanged neighbors', fontsize=18)
ax_zoom.set_xlabel('Time (s)', fontsize=18)
# ax_zoom.set_xlim(0,0.25)
# ax_zoom.set_ylim(0.6,1.1)
ax_zoom.fill_between(
    df.time[0:25],
    y1 = df.frac_sim_neighs_avg[0:25]-df.frac_sim_neighs_std[0:25],
    y2 = df.frac_sim_neighs_avg[0:25]+df.frac_sim_neighs_std[0:25], alpha=0.3, color='gray')
ax_zoom.plot(df.time[0:25],df.frac_sim_neighs_avg[0:25], '-o', color='black')
plt.show()

Time aggregated graph and hierarchical clustering

# Number of graphs to be aggregated together
n_agg_graphs = 5 # how many time frames (in the selected nodelink files) are used for each aggregation, not second in unit

times = df['time'].values
tstep = np.median(np.diff(df.time.values))
nbins = np.int(np.round(len(times)/n_agg_graphs))+1

# nbins = np.int(np.round((times.max()-times.min())/(n_agg_graphs*tstep)))+1
# df['agg_time'] = np.digitize(
#     times,
#     np.linspace(times.min(),times.max(),nbins)
# )

df['agg_index'] = np.digitize(
    range(0,len(times)),
    np.linspace(0,len(times),nbins)
)
# print(df['agg_index'])
# print(np.linspace(0,len(times),nbins))
# print([[agg_time] for agg_time, df_agg in df.groupby('agg_index')])
# print(len([[agg_time] for agg_time, df_agg in df.groupby('agg_index')])

for agg_time, df_agg in df.groupby('agg_index'):

    # Get first adjacency matrix
    adj = df_agg.graph[df_agg.index[0]].get_adjacency(attribute='length')
    
    for g in df_agg.graph.values[1:]:
        adj += g.get_adjacency(attribute='length')
        
    adj = np.array(adj.data).reshape(adj.shape)
        
    # Min and max in the weighted adj matrix
    dmax = adj.max()
    dmin = adj[adj>0].min()
        
    # Neighbors more often connected have low weight
    adj[adj>0] = np.abs( adj[adj>0] - (dmin+dmax) ).astype(np.float)
    adj = adj / n_agg_graphs
    
    # Create a graph from the aggregated adj matrix
    g_agg = igraph.Graph.Adjacency((adj > 0).tolist(), mode=igraph.ADJ_UNDIRECTED)
    g_agg.es['length'] = adj[adj.nonzero()]
    
    # Calculate shortest paths length
    spl = g_agg.shortest_paths_dijkstra(weights=g_agg.es['length'])
    distances = np.array(spl).reshape(adj.shape)

    # Replace infinities with a very large distance
    distances[np.isinf(distances)] = distances[~np.isinf(distances)].max()
    
    # Clustering
    threshold = 0.3
    fig, axs = plt.subplots(1,2,figsize=(12,4))
    linkage = hierarchy.linkage(distances, method="single")
    clusters = hierarchy.fcluster(linkage, threshold, criterion="distance")
    dend = hierarchy.dendrogram(linkage, color_threshold=threshold, ax=axs[1])
    order = dend['leaves']
    distances = distances[order,:]
    distances = distances[:,order]
    
    #After clustering, convert the largest distances (& not indirectly connected nodes) to nan and plot as white in the colormap
    max_index2 = np.where(distances == np.amax(distances)) 
    distances[max_index2] = np.nan
    current_cmap = plt.cm.get_cmap()
    current_cmap.set_bad(color='white')
 
    sc1 = axs[0].imshow(distances)
#     cbar = fig.colorbar(sc1,ax=axs[0])
    cbar = fig.colorbar(sc1, ax=axs[0],extend='neither')
#     sc1.set_clim(vmin=0,vmax=0.9)
    
    cbar.set_label(r'Adjusted frame-wise distance (nm)', fontsize=14)
    axs[0].set_title(f'Aggregate: {df_agg.time.min():.3f} to {df_agg.time.max():.3f} s')    
    axs[1].set_xlabel("Node")
#     axs[1].set_ylabel("Dissimilarity")
    plt.show()

Diffusivity as a function of cluster size

In [ ]:
df_diff = pd.DataFrame([])

for index in tqdm(df.index):
    
    g = df.graph[index]
    
    ccs = g.clusters()
    
    df_cc = pd.DataFrame([{'cluster': m, 'diffusivity': d} for (m,d) in zip(ccs.membership,g.vs['diffusivity'])])
    sizes = df_cc.groupby('cluster').size()
    df_cc = df_cc.groupby('cluster').agg(['mean','std'])
    df_cc['time'] = df.time[index]
    df_cc['nmols'] = sizes
#     print(sum(sizes))
#     print(df_cc)
    
    df_diff = pd.concat([df_diff,df_cc], axis=0, ignore_index=True)


In [ ]:
nucleation_time_guess = 0.5

cmap = plt.cm.get_cmap('jet')
fig, ax = plt.subplots(1,1, figsize=(6,4))
# sc = ax.scatter(df_diff.nmols,df_diff[('diffusivity','mean')], s=1, c=df_diff.time, cmap=cmap)
# timemax = max(df.time)
timemax = max(df.time)
timemin = max(df.time)*0
timeindex = df_diff.time[(df_diff.time >= timemin) & (df_diff.time <= timemax)].index
sc = ax.scatter(df_diff.nmols[timeindex],df_diff[('diffusivity','mean')][timeindex],s=1, c=df_diff.time[timeindex], cmap=cmap)
ax.set_xscale('log')
# ax.set_yscale('log')
plt.yscale('symlog')
ax.set_xlabel('Cluster size (#molecules)', fontsize=18)
ax.set_ylabel(r'Avg. diffusivity ($\mu m^2/s$)', fontsize=18)
# ax.set_ylabel(r'Avg. diffusivity ($\times 10^{-13}~m^2/s$)', fontsize=18)
# ax.set_xlim(0.75,20)
# ax.set_ylim(2.5,13)
cbar = plt.colorbar(sc,extend='neither')
sc.set_clim(vmin=timemin,vmax=timemax)
cbar.set_label('time (s)', fontsize=14)

# Plot the average molecular diffusion constants vs the cluster size that molecules are within
fig2, ax2 = plt.subplots(1,1, figsize=(6,4))
df_diff_timeindex = pd.DataFrame({"cluster_size": df_diff.nmols[timeindex], "Diff_cluster_size": df_diff[('diffusivity','mean')][timeindex]})
Diff_cluster_size = df_diff_timeindex.groupby('cluster_size').agg(['mean','std'])
Diff_cluster_size = Diff_cluster_size.reset_index() #Reset the dataframe to regular column
# Linear fitting the log scale graph (check power law index)
coef = np.polyfit(np.log10(Diff_cluster_size["cluster_size"]),np.log10(Diff_cluster_size[('Diff_cluster_size','mean')]),1)
poly1d_fn = np.poly1d(coef) # poly1d_fn is now a function which takes in x and returns an estimate for y
Diff_cluster_size_fit = [pow(10,i) for i in poly1d_fn(np.log10(Diff_cluster_size["cluster_size"]))]
plt.errorbar(Diff_cluster_size["cluster_size"], Diff_cluster_size[('Diff_cluster_size','mean')], yerr = Diff_cluster_size[('Diff_cluster_size','std')], xerr = None)
ax2.plot(Diff_cluster_size["cluster_size"],Diff_cluster_size_fit,'--r',label=f'Exponent $\\alpha$={round(coef[0], 2)}')
ax2.legend(loc=0)
#ax2.text(10, 0.4,)
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.set_xlabel('Cluster size (#molecules)', fontsize=18)
ax2.set_ylabel(r'Avg. diffusivity ($\mu m^2/s$)', fontsize=18)

# Plot the average cluster size vs time
fig3, ax3 = plt.subplots(1,1, figsize=(6,4))
cluster_timeindex = pd.DataFrame({"time": df_diff.time[timeindex],"cluster_size": df_diff.nmols[timeindex]})
cluster_time = cluster_timeindex.groupby("time").agg(['mean','std'])
cluster_time = cluster_time.reset_index() #Reset the dataframe to regular column
# Linear fitting the log scale graph (check power law index)
coef = np.polyfit(np.log10(cluster_time["time"]),np.log10(cluster_time[('cluster_size','mean')]),1)
poly1d_fn = np.poly1d(coef) # poly1d_fn is now a function which takes in x and returns an estimate for y
cluster_time_fit = [pow(10,i) for i in poly1d_fn(np.log10(cluster_time["time"]))]
plt.errorbar(cluster_time["time"], cluster_time[('cluster_size','mean')], yerr = cluster_time[('cluster_size','std')], xerr = None)

ax3.plot(cluster_time['time'],cluster_time_fit,'--r',label=f'Total: Exponent $\\alpha$={round(coef[0], 2)}')

first_select = np.where( cluster_time["time"] < nucleation_time_guess)[0]
coef = np.polyfit(np.log10(cluster_time["time"][first_select]),np.log10(cluster_time[('cluster_size','mean')][first_select]),1)
poly1d_fn = np.poly1d(coef)
cluster_time_fit = [pow(10,i) for i in poly1d_fn(np.log10(cluster_time["time"]))]
ax3.plot(cluster_time['time'],cluster_time_fit,'--g',label=f'First: Exponent $\\alpha$={round(coef[0], 2)}')


ax3.legend(loc=0)
#ax3.text(0.05, 10, f'Exponent $\\alpha$={round(coef[0], 2)}')
ax3.set_xscale('log')
ax3.set_yscale('log')
ax3.set_xlabel('time / s', fontsize=18)
ax3.set_ylabel(r'Avg. cluster size (#molecules)', fontsize=18)

plt.show()

### Added Cell 2: Replot the cluster diffusivity with cluster size by combining all time information and use single color for each dots but also add mean(not median) and std values with dots on the background

In [ ]:
## Combined 1st and 2nd figure of Deff vs. cluster size without time information as colormap
cmap = plt.cm.get_cmap('jet')
fig, ax = plt.subplots(1,1, figsize=(6,4))
timemax = max(df.time)
timemin = max(df.time)*0
timeindex = df_diff.time[(df_diff.time >= timemin) & (df_diff.time <= timemax)].index
sc = ax.scatter(df_diff.nmols[timeindex],df_diff[('diffusivity','mean')][timeindex], s=1, c='grey')

# Plot the average molecular diffusion constants vs the cluster size that molecules are within
df_diff_timeindex = pd.DataFrame({"cluster_size": df_diff.nmols[timeindex], "Diff_cluster_size": df_diff[('diffusivity','mean')][timeindex]})
Diff_cluster_size = df_diff_timeindex.groupby('cluster_size').agg(['mean','std'])
Diff_cluster_size = Diff_cluster_size.reset_index() #Reset the dataframe to regular column

plt.errorbar(Diff_cluster_size["cluster_size"], Diff_cluster_size[('Diff_cluster_size','mean')], yerr = Diff_cluster_size[('Diff_cluster_size','std')], xerr = None)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('Cluster size (#molecules)', fontsize=18)
ax.set_ylabel(r'Avg. diffusivity ($\mu m^2/s$)', fontsize=18)
ax.set_ylim(1e-5,10)

plt.show()